In [14]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [ ]:
from torchvision import datasets, transforms
import torch

data_path = '../data/ch7/'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.ToTensor() 
)

images = torch.stack([img for img, _ in cifar10], dim=3)

mean = images.view(3, -1).mean(dim=1)
std = images.view(3, -1).std(dim=1)

print("Mean (R, G, B):", mean)
print("Std (R, G, B):", std)

Mean (R, G, B): tensor([0.4914, 0.4822, 0.4465])
Std (R, G, B): tensor([0.2470, 0.2435, 0.2616])


In [ ]:
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

In [9]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

cifar2_val = [(img, label_map[label]) 
              for img, label in cifar10_val
              if label in [0, 2]]

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.543558
Epoch: 1, Loss: 0.436781
Epoch: 2, Loss: 0.369581
Epoch: 3, Loss: 0.525758
Epoch: 4, Loss: 0.380984
Epoch: 5, Loss: 0.403744
Epoch: 6, Loss: 0.343644
Epoch: 7, Loss: 0.523498
Epoch: 8, Loss: 0.533998
Epoch: 9, Loss: 0.469928
Epoch: 10, Loss: 0.545019
Epoch: 11, Loss: 0.323323
Epoch: 12, Loss: 0.413933
Epoch: 13, Loss: 0.323450
Epoch: 14, Loss: 0.392169
Epoch: 15, Loss: 0.381515
Epoch: 16, Loss: 0.240899
Epoch: 17, Loss: 0.400147
Epoch: 18, Loss: 0.333209
Epoch: 19, Loss: 0.402492
Epoch: 20, Loss: 0.358327
Epoch: 21, Loss: 0.117419
Epoch: 22, Loss: 0.272192
Epoch: 23, Loss: 0.201518
Epoch: 24, Loss: 0.073683
Epoch: 25, Loss: 0.166854
Epoch: 26, Loss: 0.109931
Epoch: 27, Loss: 0.376480
Epoch: 28, Loss: 0.272744
Epoch: 29, Loss: 0.055994
Epoch: 30, Loss: 0.154708
Epoch: 31, Loss: 0.326919
Epoch: 32, Loss: 0.192552
Epoch: 33, Loss: 0.244193
Epoch: 34, Loss: 0.070251
Epoch: 35, Loss: 0.081186
Epoch: 36, Loss: 0.090806
Epoch: 37, Loss: 0.145870
Epoch: 38, Loss: 0.284

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.999600


In [12]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.812000
